In [1]:
import pandas as pd
import scipy as sparse
from sklearn.metrics.pairwise import cosine_similarity 


In [7]:
data= pd.read_csv('/Users/akriti/Desktop/development/Datasets/toy_movie_dataset.csv',index_col=0)
data= data.fillna(0)
data

,action1,action2,action3,Romantic1,Romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [8]:
def standarized(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row
data_std=data.apply(standarized)
data_std

,action1,action2,action3,Romantic1,Romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [9]:
#item -to- item filtering 
# cosine calculates the similarity between rows that is the reason why we have done transpose as we want similarity between items and not users
item_similarity = cosine_similarity(data_std.T)
print(item_similarity)


[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [10]:
# converting numpy array into a DataFrame 
item_similarity_df= pd.DataFrame(item_similarity,index=data.columns,columns=data.columns)
item_similarity_df

,action1,action2,action3,Romantic1,Romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
Romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
Romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [15]:
# recommendation :

def get_similar_movie(movie_name,user_rating):
    similar_score= item_similarity_df[movie_name]*(user_rating-2.5)
    # 2.5 is the mean of ratingsThe reason of doing this is if the user gives romantic movie then the function was initially returning the same value 
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

print(get_similar_movie('Romantic1',1))
    

action3      1.271919
action2      1.267731
action1      1.199116
Romantic2   -0.222059
romantic3   -1.085620
Romantic1   -1.500000
Name: Romantic1, dtype: float64


In [19]:
# to take multiple user rating 

action_l =[('action1',5),("Romantic1",1),("romantic3",1)]
similar_movies= pd.DataFrame()
for movie,rating in action_l:
    similar_movies=similar_movies.append(get_similar_movie(movie,rating),ignore_index=True)
similar_movies.head() 
similar_movies.sum().sort_values(ascending=False)

action1      5.070275
action3      4.509394
action2      4.299514
Romantic2   -0.876447
Romantic1   -4.584147
romantic3   -4.870886
dtype: float64